In [1]:
import pandas as pd

## Lecture des données

In [51]:
data = pd.read_csv("finisseurs.csv",sep=";",decimal=",",index_col=0)
epdeg = pd.read_csv("epdeg.csv",sep=";",decimal=",",index_col=0)
data = pd.concat([data,epdeg],axis=1)
data = data[data['metal']==4]
data.dropna(inplace=True)



In [ ]:
data.groupby(['metal']).size().sort_values()

In [52]:
data = data[(data['esort_1']>=16) & (data['esort_1']<=35)]
data = data[(data['esort_2']>=5) & (data['esort_2']<=35)]
data = data[(data['esort_3']>=1) & (data['esort_3']<=35)]
data = data[(data['esort_4']>=1) & (data['esort_4']<=35)]
data = data[(data['esort_5']>=1) & (data['esort_5']<=35)]
data = data[(data['esort_6']>=1) & (data['esort_6']<=35)]
data = data[(data['esort_7']>=1) & (data['esort_7']<=35)]

for p in [1,2,3,4,5,6,7]:
    data = data[data['pf'+str(p)]>0]
data = data[data['larg']>0]


In [53]:

for p in [1,2,3,4,5,6,7]:
    data['pf'+str(p)] = data['pf'+str(p)]/1000000

#for p in [2,3,4,5,6,7]:
#    data['epentr_'+str(p)] = data['esort_'+str(p-1)]

In [54]:
data.head()

,pf1,pf2,pf3,pf4,pf5,pf6,pf7,metal,larg,esort_1,esort_2,esort_3,esort_4,esort_5,esort_6,esort_7,epentr_1
Date,,,,,,,,,,,,,,,,,
01/01/2019 01:19:24,7.289175,7.428041,6.730653,7.056374,6.950488,6.168378,2.372457,4,1560.737305,19.875496,12.323304,8.256593,5.715446,4.297836,3.329766,3.00,34.026127
01/01/2019 01:20:44,7.836250,7.648174,7.337168,7.625006,7.238607,6.410951,2.302351,4,1514.797119,19.439274,11.856285,7.855474,5.375022,4.009247,3.073996,2.75,34.028179
01/01/2019 01:23:15,6.211434,8.151202,7.205716,6.917541,7.316759,6.429595,2.136229,4,1463.295898,20.079947,11.247037,7.313984,4.964491,3.655787,2.787378,2.50,34.030140
01/01/2019 01:24:36,5.396618,7.666709,6.767735,6.407474,6.441339,5.923056,1.879329,4,1463.044434,20.103851,11.270432,7.330319,4.972833,3.660301,2.788233,2.50,34.030140
01/01/2019 01:26:17,6.435756,8.778601,7.812439,7.176302,7.097075,6.686953,2.144244,4,1464.864746,20.099497,11.268517,7.324177,4.970414,3.659725,2.788233,2.50,34.030140


In [62]:
from sklearn.linear_model import LinearRegression

dico_model = dict()

dico_model['model_f1'] = LinearRegression().fit(X=data[['larg','epentr_1','esort_1']],y=data['pf1'])
dico_model['model_f2'] = LinearRegression().fit(X=data[['larg','esort_1','esort_2']],y=data['pf2'])
dico_model['model_f3'] = LinearRegression().fit(X=data[['larg','esort_2','esort_3']],y=data['pf3'])
dico_model['model_f4'] = LinearRegression().fit(X=data[['larg','esort_3','esort_4']],y=data['pf4'])
dico_model['model_f5'] = LinearRegression().fit(X=data[['larg','esort_4','esort_5']],y=data['pf5'])
dico_model['model_f6'] = LinearRegression().fit(X=data[['larg','esort_5','esort_6']],y=data['pf6'])
dico_model['model_f7'] = LinearRegression().fit(X=data[['larg','esort_6','esort_7']],y=data['pf7'])


In [90]:
dico_model_coefs = dict()


for mod in dico_model.keys():
    fact_name  = dico_model[mod].feature_names_in_.tolist()
    coef_value = dico_model[mod].coef_
    dico_coef  = {f: c for f,c in zip(fact_name,coef_value)}
    dico_coef['intercept'] = dico_model[mod].intercept_
    dico_model_coefs[mod] = dico_coef

In [92]:
import pickle 
  
  
try: 
    geeky_file = open('dico_model_coefs', 'wb') 
    pickle.dump(dico_model_coefs, geeky_file) 
    geeky_file.close() 
  
except: 
    print("Something went wrong")

In [84]:
dico_coef_global = {key: 0.0 for key in list_all_var}
dico_coef_global['intercept'] = 0.0

In [85]:

for mod in dico_model.keys():
    fact_name  = dico_model[mod].feature_names_in_.tolist()
    coef_value = dico_model[mod].coef_
    #dico_coef = {f: c for f,c in zip(fact_name,coef_value)}

    for f,c in zip(fact_name,coef_value):
        dico_coef_global[f] +=  c
    dico_coef_global['intercept'] += dico_model[mod].intercept_


In [87]:
import pickle 
  
  
try: 
    geeky_file = open('dico_coef', 'wb') 
    pickle.dump(dico_coef_global, geeky_file) 
    geeky_file.close() 
  
except: 
    print("Something went wrong")

In [88]:
data.to_pickle('data.pkl')

In [61]:
model_f5.score(X=data[['larg','esort_4','esort_5']],y=data['pf5'])

0.8150145720906254